# Generate knowledge tuning data

## Overview

This notebook demonstrates how you can generate high-quality knowledge tuning datasets using the Red Hat AI Synthetic Data Generation (SDG) Hub framework. It creates multiple types of document augmentations and corresponding question and answer (Q&A) pairs that you can use to train or fine-tune language models for enhanced summarization and knowledge extraction capabilities.

This notebook generates the following types of knowledge tuning datasets:
- **Extractive summaries**: Concise summaries that extract key information directly from source documents.
- **Detailed summaries**: Coprehensive summaries that provide thorough coverage of document content.
- **Key facts**: Structured fact extraction with corresponding Q&A pairs.
- **Document-based Q&A**: Question-answer pairs generated directly from document content.

This notebook outputs structured training data. For each augmentation, it saves a JSONL dataset. 

## Prerequisites

- You completed the [Data preprocessing ](../03_Knowledge_Generation/Knowledge_Generation.ipynb) notebook.

- The following environment variables are set in the [.env.example](./.env.example) file:
  - `TEACHER_MODEL_API_KEY` — The LLM API key.
  - `TEACHER_MODEL_BASE_URL` — The LLM HTTP endpoint.
  - `TEACHER_MODEL_NAME` — The model to call for data generation.

## Install dependencies

⚠️ NOTE: If you use vllm to serve your teacher model, edit the following Python command to include the vllm option:

`pip install -qqU .[vllm]`

In [ ]:
!pip install -qqU .

## Set up paths and directories

In [ ]:
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

WORKSPACE = Path.cwd().parent  # Path to the workspace directory

OUTPUT_DIR = WORKSPACE / "output" / "step_03"

OUTPUT_DIR.mkdir(
    parents=True, exist_ok=True
)  # Create the output directory if it doesn't exist

# Create the output directory
dataset_output_folders = [
    "extractive_summary",
    "detailed_summary",
    "key_facts_to_qa",
    "document_based_qa",
]
for dataset_folder in dataset_output_folders:
    dataset_output_dir = OUTPUT_DIR / dataset_folder
    dataset_output_dir.mkdir(parents=True, exist_ok=True)
    print(f"✅ Created output directory for {dataset_folder}")

## Configure the teacher model

You can use your own inference server for the teacher model. You can also host your own teacher model by using `vllm` in a seperate terminal and then use the available GPUs.

🚨 The following example command runs the inference server by using vllm. Refer to the https://docs.redhat.com/en/documentation/red_hat_ai_inference_server(vllm documentation) for information about configuring parameters.


```bash
vllm serve RedHatAI/gpt-oss-120b \
    --host 0.0.0.0 \
    --port 8000 \
    --served-model-name teacher \
    --tensor-parallel-size=8 \
    --distributed-executor-backend=mp \
    --max-model-len=128000 \
```

To view the supported teacher models for the SDG Hub flow, use the `flow.get_model_recommendations()` command.

In [ ]:
MODEL_NAME = os.getenv("TEACHER_MODEL_NAME", "openai/gpt-oss-120b")
API_KEY = os.getenv("TEACHER_MODEL_API_KEY", "None")  # Provide your API key here
ENDPOINT = os.getenv("TEACHER_MODEL_BASE_URL", "http://0.0.0.0:8000/v1")

print(f"MODEL NAME : {MODEL_NAME}")
print(f"API_KEY : {API_KEY}")
print(f"ENDPOINT : {ENDPOINT}")

## Configure SDG

In [ ]:
# Required to run the flow with async mode
import nest_asyncio

nest_asyncio.apply()


RUN_ON_VALIDATION_SET = True  # Whether to run the model on the validation set
SEED_DATA_SUBSAMPLE = 0  # Subsample the seed data for faster testing.

SEED_DATA_FILE = (
    WORKSPACE / "output" / "step_02" / "seed_data.jsonl"
)  # Path to the seed data file generated in step 2


ENABLE_REASONING = False  # Whether to enable reasoning in the model
NUMBER_OF_SUMMARIES = 50  # Number of summaries to generate
MAX_CONCURRENCY = 50  # Maximum number of concurrent requests to the model

# Set the timeout for the teacher model inference server used by sdg flows. Default: 120
INFERENCE_SERVER_TIMEOUT = 120


os.environ["LITELLM_REQUEST_TIMEOUT"] = str(INFERENCE_SERVER_TIMEOUT)
if not SEED_DATA_FILE.exists():
    raise FileNotFoundError(
        f"\nNot a valid seed data ! {SEED_DATA_FILE}.\nPlease run step 2 to generate the seed data. \n(or) Provide the correct path to the seed data file."
    )

In [ ]:
# Third Party

# First Party
from sdg_hub import Flow, FlowRegistry

In [ ]:
# Load the seed data generated when you ran the Data Processing notebook
quality_corpus = pd.read_json(SEED_DATA_FILE, lines=True)

if SEED_DATA_SUBSAMPLE > 0:
    quality_corpus = quality_corpus.iloc[:SEED_DATA_SUBSAMPLE]

## Run SDG

Run the following code to create a knowledge flow from the provided YAML file. For the purposes of this example, the dataset is small.

For large scale generation, use the python command provided in the next cell.
You can analyze the generated data to check that the quality is similar to the provided Q&A pairs.

#### Discover the available generation flows

In [ ]:
# Auto-discover all available flows (no set up needed!)
FlowRegistry.discover_flows()

# List the available flows
flows = FlowRegistry.list_flows()
print(f"Available flows: {flows}")

# Search the flows by tag
qa_flows = FlowRegistry.search_flows(tag="question-generation")
print(f"QA flows: {qa_flows}")

### Generate the extractive summaries dataset

In [ ]:
# Generate data for extractive summary
flow_name = "Extractive Summary Knowledge Tuning Dataset Generation Flow"
flow_path = FlowRegistry.get_flow_path(flow_name)
flow = Flow.from_yaml(flow_path)

# Set model configuration
flow.set_model_config(
    model=MODEL_NAME,
    api_base=ENDPOINT,
    api_key=API_KEY,
)

# Generate data for extractive summary
if ENABLE_REASONING:
    # Increase max tokens to accommodate reasoning content
    runtime_params = {
        "question_generation": {"max_tokens": 1024},
        "gen_extractive_summary": {"n": NUMBER_OF_SUMMARIES, "max_tokens": 6000},
    }
else:
    runtime_params = {"gen_extractive_summary": {"n": NUMBER_OF_SUMMARIES}}

extractive_summary_generated_data = flow.generate(
    quality_corpus, runtime_params=runtime_params, max_concurrency=MAX_CONCURRENCY
)

extractive_summary_generated_data.to_json(
    os.path.join(OUTPUT_DIR, "extractive_summary", "gen.jsonl"),
    orient="records",
    lines=True,
)

print(f"✓ Extractive summary: {len(extractive_summary_generated_data)} records")

print(f"✓ Columns: {list(extractive_summary_generated_data.columns)}")

### Generate the detailed summary dataset

In [ ]:
# Generate similar data for Detailed Summary
flow_name = "Detailed Summary Knowledge Tuning Dataset Generation Flow"
flow_path = FlowRegistry.get_flow_path(flow_name)
flow = Flow.from_yaml(flow_path)

# Set model configuration
flow.set_model_config(
    model=MODEL_NAME,
    api_base=ENDPOINT,
    api_key=API_KEY,
)

if ENABLE_REASONING:
    # Increase max tokens to accommodate reasoning content
    runtime_params = {
        "question_generation": {"max_tokens": 1024},
        "gen_detailed_summary": {"n": NUMBER_OF_SUMMARIES, "max_tokens": 6000},
    }
else:
    runtime_params = {"gen_detailed_summary": {"n": NUMBER_OF_SUMMARIES}}
# Generate data for detailed summary
detailed_summary_generated_data = flow.generate(
    quality_corpus, runtime_params=runtime_params, max_concurrency=MAX_CONCURRENCY
)

detailed_summary_generated_data.to_json(
    os.path.join(OUTPUT_DIR, "detailed_summary", "gen.jsonl"),
    orient="records",
    lines=True,
)

print(f"✓ Detailed summary: {len(detailed_summary_generated_data)} records")

print(f"✓ Columns: {list(detailed_summary_generated_data.columns)}")

### Generate the key facts dataset

In [ ]:
# Generate similar data for key facts
flow_name = "Key Facts Knowledge Tuning Dataset Generation Flow"
flow_path = FlowRegistry.get_flow_path(flow_name)
flow = Flow.from_yaml(flow_path)

# Set model configuration
flow.set_model_config(
    model=MODEL_NAME,
    api_base=ENDPOINT,
    api_key=API_KEY,
)

runtime_params = {}

if ENABLE_REASONING:
    # Increase max tokens for Question Generation to accommodate reasoning content
    runtime_params = {
        "generate_key_fact_qa": {"max_tokens": 6000},
    }

# Generate data for key facts summary
key_facts_generated_data = flow.generate(
    quality_corpus, runtime_params=runtime_params, max_concurrency=MAX_CONCURRENCY
)

key_facts_generated_data.to_json(
    os.path.join(OUTPUT_DIR, "key_facts_to_qa", "gen.jsonl"),
    orient="records",
    lines=True,
)

print(f"✓ Key facts: {len(key_facts_generated_data)} records")

print(f"✓ Columns: {list(key_facts_generated_data.columns)}")

### Generate the document-based dataset

In [ ]:
flow_name = "Document Based Knowledge Tuning Dataset Generation Flow"
flow_path = FlowRegistry.get_flow_path(flow_name)
flow = Flow.from_yaml(flow_path)

# Set model configuration
flow.set_model_config(
    model=MODEL_NAME,
    api_base=ENDPOINT,
    api_key=API_KEY,
)

runtime_params = {}

if ENABLE_REASONING:
    # Increase max tokens to accommodate reasoning content
    runtime_params = {
        "question_generation": {"max_tokens": 2048},
    }

document_based_generated_data = flow.generate(
    quality_corpus, runtime_params=runtime_params, max_concurrency=MAX_CONCURRENCY
)

document_based_generated_data.to_json(
    os.path.join(OUTPUT_DIR, "document_based_qa", "gen.jsonl"),
    orient="records",
    lines=True,
)

print(f"✓ Document based: {len(document_based_generated_data)} records")

print(f"✓ Columns: {list(document_based_generated_data.columns)}")

🎉 You now have all four of the document augmentations (detailed summaries, extractive summaries, key facts and document-based) along with their corresponding Q&A pairs.

## Next step
  
   In the [Knowledge mixing notebook](../04_Knowledge_Mixing/Knowledge_Mixing.ipynb), you combine and curate these datasets to prepare your final training data.